In [ ]:
%config Completer.use_jedi = False
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
# Register a model:
# from azureml.core.model import Model
# model = Model.register(ws, model_name="damageDetector", model_path="damage_detector_weights.h5")

In [ ]:
from azureml.core.model import Model

model = Model(ws, 'damageDetector')

In [ ]:
# Setup Environment
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='myenv')
python_packages = ['numpy', 'tensorflow==2.1.0', 'h5py==2.10.0','image','azureml-contrib-services']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(environment=env, entry_script='./score_damage.py')

In [ ]:
# Deployment
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import LocalWebservice
dep_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 4)
# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "service2",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = dep_config,
                      )
service.wait_for_deployment(show_output=True)

In [ ]:
import requests
import PIL
import json

# Test
img = open('car-damaged.jpg', 'rb').read()
headers = {'Content-Type':'application/json'}

response = requests.post(service.scoring_uri,data=img, headers=headers)

print(json.loads(response.text))


In [ ]:
service.get_logs().split('\n')